In [ ]:
# Подготовка CSV
    csv_file = open('avito_reviews_msc_7.csv', 'w', newline='', encoding='utf-8')
    writer = csv.DictWriter(csv_file, fieldnames=["developer","username","review_date","rating","project_name","summary","advantages","disadvantages"])
    writer.writeheader()

    # Ввод ссылок
    raw = input("Введите ссылки застройщиков через запятую:\n")
    urls = [u.strip() for u in raw.split(',') if u.strip()]

    for base_url in urls:
        dev = extract_developer_from_url(base_url) or "unknown_developer"
        print(f"Парсим застройщика: {dev}")
        # стартовая задержка
        t0 = random.uniform(INITIAL_WAIT_DEV_MIN, INITIAL_WAIT_DEV_MAX)
        print(f"  стартовая пауза {t0:.1f}s")
        time.sleep(t0)

        # пагинация
        parsed = urlparse(base_url)
        page = int(parse_qs(parsed.query).get('reviewsPage', ['1'])[0])
        while True:
            url = build_page_url(base_url, page)
            print(f"  страница {page}: {url}")
            driver.get(url)
            time.sleep(random.uniform(MIN_DELAY_PAGES, MAX_DELAY_PAGES))
            reviews = parse_reviews(driver.page_source)
            if not reviews:
                print(f"  нет отзывов на странице {page}, выходим")
                break
            for r in reviews:
                r['developer'] = dev
                writer.writerow(r)
            print(f"  сохранено {len(reviews)} отзывов")
            page += 1

        # пауза между застройщиками
        tb = random.uniform(MIN_DELAY_DEVS, MAX_DELAY_DEVS)
        print(f"Ждём {tb:.1f}s перед следующим застройщиком")
        time.sleep(tb)

    csv_file.close()
    driver.quit()
    print("Готово: avito_reviews_msc_7.csv создан.")

if name == '__main__':
    main()